In [1]:
import cv2
from keras.models import load_model
import numpy as np
import os

In [2]:
model = load_model('emojinator.h5')

In [3]:
def get_emojis():
    emojis_folder = 'hand_emo/'
    emojis = []
    for emoji in range(len(os.listdir(emojis_folder))):
        print(emoji)
        emojis.append(cv2.imread(emojis_folder+str(emoji)+'.png', -1))
    return emojis

In [4]:
def keras_predict(model, image):
    processed = keras_process_image(image)
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class


def keras_process_image(img):
    image_x = 50
    image_y = 50
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

In [5]:
def overlay(image, emoji, x,y,w,h):
    emoji = cv2.resize(emoji, (w, h))
    try:
        image[y:y+h, x:x+w] = blend_transparent(image[y:y+h, x:x+w], emoji)
    except:
        pass
    return image

def blend_transparent(face_img, overlay_t_img):
    # Split out the transparency mask from the colour info
    overlay_img = overlay_t_img[:,:,:3] # Grab the BRG planes
    overlay_mask = overlay_t_img[:,:,3:]  # And the alpha plane

    # Again calculate the inverse mask
    background_mask = 255 - overlay_mask

    # Turn the masks into three channel, so we can use them as weights
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    # Create a masked out face image, and masked out overlay
    # We convert the images to floating point in range 0.0 - 1.0
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    # And finally just add them together, and rescale it back to an 8bit integer image
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))

In [ ]:
emojis = get_emojis()
cap = cv2.VideoCapture(0)
x, y, w, h = 300, 50, 350, 350

while (cap.isOpened()):
    ret, img = cap.read()
    img = cv2.flip(img, 1)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask2 = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255]))
    res = cv2.bitwise_and(img, img, mask=mask2)
    gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
    median = cv2.GaussianBlur(gray, (5, 5), 0)

    kernel_square = np.ones((5, 5), np.uint8)
    dilation = cv2.dilate(median, kernel_square, iterations=2)
    opening = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel_square)
    ret, thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)

    thresh = thresh[y:y + h, x:x + w]
    contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]
    if len(contours) > 0:
        contour = max(contours, key=cv2.contourArea)
        if cv2.contourArea(contour) > 2500:
            x, y, w1, h1 = cv2.boundingRect(contour)
            newImage = thresh[y:y + h1, x:x + w1]
            newImage = cv2.resize(newImage, (50, 50))
            pred_probab, pred_class = keras_predict(model, newImage)
            print(pred_class, pred_probab)
            img = overlay(img, emojis[pred_class], 400, 250, 90, 90)

    x, y, w, h = 300, 50, 350, 350
    cv2.imshow("Frame", img)
    cv2.imshow("Contours", thresh)
    k = cv2.waitKey(10)
    if k == 27:
        break

0
1
2
3
4
5
6
7
8
9
10
11
12
9 0.9970204
6 0.90934277
6 0.9348011
6 0.9348011
6 0.9734778
9 0.5201023
9 0.6749253
6 0.74609566
9 0.7350473
9 0.6586405
9 0.46163568
9 0.6242879
5 0.88695955
9 0.7935543
9 0.9722316
9 0.94363314
9 0.7680836
9 0.86223316
9 0.87671214
9 0.88680303
9 0.84790164
4 0.49091032
9 0.7264937
11 0.44005767
9 0.9906268
9 0.74740994
4 0.40141097
10 0.9750199
9 0.8358713
4 0.68205667
9 0.74061054
9 0.74061054
4 0.79671824
9 0.6019114
4 0.72966737
9 0.91103834
9 0.69964224
9 0.3474922
9 0.42268202
10 0.9697408
9 0.8644977
9 0.6068352
6 0.49636623
9 0.86905813
5 0.76124525
5 0.76124525
9 0.97084
9 0.98723024
9 0.9955747
9 0.5475808
9 0.63540167
9 0.5767839
9 0.9321347
4 0.41900244
9 0.99613047
9 0.76639783
9 0.76639783
9 0.5943251
9 0.82373285
9 0.8247402
9 0.8356426
9 0.6918499
9 0.68526435
9 0.6726458
9 0.47646335
6 0.507222
9 0.63485885
6 0.53280646
9 0.47717273
9 0.8152116
9 0.8152116
6 0.58337694
9 0.38661927
6 0.6477771
9 0.9946603
9 0.98273385
9 0.99325633
9 0.89

9 0.9977841
9 0.99949396
9 0.9989231
9 0.9998031
6 0.9990251
6 0.98867214
6 0.9968161
6 0.9968161
10 0.946279
9 0.9986008
9 0.9998673
4 0.9226857
9 0.9131388
9 0.9131388
9 0.8578963
9 0.92504025
9 0.98648155
10 0.94374907
9 0.85844344
9 0.9999628
10 0.5752742
9 0.98459065
9 0.8177574
9 0.9271133
9 0.99325114
9 0.9966055
9 0.98760545
9 0.99383813
9 0.982053
9 0.982053
9 0.9991117
9 0.99992967
9 0.9985422
9 0.9820832
9 0.9982426
9 0.9982426
9 0.9949234
9 0.9999505
9 0.99986744
9 0.9985422
9 0.9963257
9 0.9975515
9 0.99974734
9 0.9990074
9 0.9983108
9 0.9852657
9 0.9852657
9 0.9998586
9 0.99994993
9 0.9996039
9 0.99867517
9 0.91427374
9 0.99912935
9 0.99957734
9 0.99877113
9 0.9996183
9 0.9999722
9 0.9997335
9 0.99929655
9 0.9990759
9 0.9990759
9 0.9998349
9 0.9996517
9 0.99981064
9 0.9999819
9 0.98709935
9 0.99697745
9 0.9996018
9 0.91562027
11 0.92509174
11 0.58802885
1 0.9902776
1 0.9964675
1 0.9865197
9 0.57261705
9 0.9816257
9 0.9999975
9 0.9999975
9 0.99993885
9 0.9997625
9 0.997517